In [1]:
import pandas as pd
import psycopg2

In [ ]:
try:
    # пытаемся подключиться к базе данных
    connect = psycopg2.connect(dbname='db1', 
                            user='test_user', 
                            password='j2M{CnnFq@',
                            port=6432,
                            host='rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net')
    print("Connection to PostgreSQL DB successful")
except:
    # в случае сбоя подключения будет выведено сообщение в STDOUT
    print('Mistake')
query_1 = "select * from test.chat_messages;"
chat_messages = pd.read_sql_query(query_1, connect)
query_2 = "select * from test.managers;"
managers = pd.read_sql_query(query_2, connect)
cursor.close() # закрываем курсор
conn.close() # закрываем соединение

In [15]:
# создаем первую таблицу со временем входящих сообщений
users_message = chat_messages[chat_messages.type == "incoming_chat_message"]
users_message = users_message[['entity_id', 'created_at']].groupby('entity_id').min('created_at')
# создаем вторую таблицу с ответами менеджеров
managers_answers = chat_messages[chat_messages.type == "outgoing_chat_message"][['entity_id', 'created_at']]
# делаем фильтрацию времени ответа менеджеров только после сообщений пользователей
managers_answers = pd.merge(managers_answers, users_message, on=['entity_id'], how='inner')
managers_answers = managers_answers[managers_answers['created_at_x'] > managers_answers['created_at_y']]
managers_answers = managers_answers[['entity_id', 'created_at_x']]
managers_answers = managers_answers.rename(columns={'created_at_x': 'created_at'})
# делаем группировку по номеру обращения и ищем время первого ответа
managers_answers = managers_answers.groupby(['entity_id'])['created_at'].min().reset_index()
# добавляем id менеджера
buf_table = chat_messages[chat_messages.type == "outgoing_chat_message"][['entity_id', 'created_by', 'created_at']]
managers_answers = pd.merge(managers_answers, buf_table, on=['created_at', 'entity_id'], how='inner')
managers_answers = managers_answers[['entity_id', 'created_by', 'created_at']]
# меняем формат времени
managers_answers['created_at'] = pd.to_datetime(managers_answers['created_at'], unit='s')
users_message['created_at'] = pd.to_datetime(users_message['created_at'], unit='s')
# соединяем время получения сообщения и ответа в одну таблицу
union_table = pd.merge(managers_answers, users_message, on='entity_id', how='inner')


In [16]:
# функция для подсчета количества времени затрачиваемого на ответы пользователей
def calculate_response_time(row):
    received = row['created_at_y']
    answered = row['created_at_x']
    
    work_start = received.replace(hour=9, minute=30, second=0, microsecond=0)
    work_end = received.replace(hour=0, minute=0, second=0, microsecond=0) + pd.Timedelta(days=1)
    
    # первое условие для не рабочего времени
    if received <= work_start and answered <= work_start:
        return pd.Timedelta(0)
           # второе условие, если сообщения пришло в не рабочее время
    elif received < work_start:
        if answered <= work_end:
            return answered - received - (work_start - received)
        elif answered > work_start + pd.Timedelta(days=1):
            return answered - received - pd.Timedelta(hours=9, minutes=30, seconds=0, microseconds=0) - (work_start - received)
        elif answered > work_end:
            return work_end - received - (work_start - received)
            # третье условие, если сообщения пришло в рабочее время
    elif received >= work_start:
        if answered <= work_end:
            return answered - received 
        elif answered > work_start + pd.Timedelta(days=1):
            return answered - received - pd.Timedelta(hours=9, minutes=30, seconds=0, microseconds=0)
        elif answered > work_end:
            return work_end - received
# применяем функцию для dataframe
union_table['time_difference'] = union_table.apply(calculate_response_time, axis=1)

In [19]:
# соединяем с таблицей менеджеров
union_table = union_table.rename(columns={'created_by': 'mop_id'})
result = pd.merge(union_table, managers, on='mop_id')
# группируем по менеджерам и ищем среднее время ответа
result = result.groupby(['mop_id', 'name_mop'])['time_difference'].mean().reset_index()
# выводим результат
result[['name_mop', 'time_difference']].sort_values(by = 'time_difference').reset_index(drop=True)

,name_mop,time_difference
0,Даша и Даша,0 days 00:02:25.480000
1,Порхачева Полина,0 days 00:04:46.750000
2,Настя и Даша,0 days 00:05:33.980000
3,Даша и Карина,0 days 00:05:41.694444444
4,Полина Мирзоян,0 days 00:07:32.400000
5,Ангелина Милованова,0 days 00:07:36.307692307
6,Лиза и Ева,0 days 00:08:35.466101694
7,Настя и Саша,0 days 00:10:06.822222222
8,Влада и Настя,0 days 00:11:11.600000
9,Мария и Соня,0 days 00:14:51.846153846
